# BIO INFORMATICS PART 2 : NETWORK MEDICINE PROJECT GROUP- 7
-  VIKRANTH ALE [1873995]
-  KENTARO KATO [1851049]


#### Disgenet Pathology Code: C0026998
#### Disgenet Pathology Name: Acute Myeloid Leukemia, M1

In [1]:
"""Loading required libraries """
import pandas as pd
import numpy as np
import re
from tqdm import trange,tqdm_notebook
import os
import warnings
warnings.filterwarnings('ignore')

#### 1.1 a) Explore the DisGeNet dataset, find the disease of interest and get the list of human genes involved

In [2]:
# Curated gene-disease associations data from DisGeNet with corresponding Uniprot entries

df = pd.read_csv("C0026998_disease_gda_summary_CURATED.tsv", sep='\t')
df.to_csv('Acute_Myeloid_Leukemia_M1.csv', encoding='utf8',index=False)

print('Total number of observations in our dataset {}'.format(df.shape))

print('')
#### Disease of Interest

print("The Disease we are interested for our analysis is {}".format(df['Disease'][0]))

df.head()


Total number of observations in our dataset (107, 18)

The Disease we are interested for our analysis is Acute Myeloid Leukemia, M1


,Disease,Disease_id,Gene,Gene_id,UniProt,Gene_Full_Name,Protein_Class,N_diseases_g,DSI_g,DPI_g,pLI,Score_gda,EL_gda,EI_gda,N_PMIDs,N_SNPs_gda,First_Ref,Last_Ref
0,"Acute Myeloid Leukemia, M1",C0026998,FLT3,2322,P36888,fms related tyrosine kinase 3,NaN,21,0.503,0.517,6.136100e-01,0.52,no reported evidence,1.0,13,0,2002.0,2017.0
1,"Acute Myeloid Leukemia, M1",C0026998,NPM1,4869,P06748,nucleophosmin 1,chaperone,13,0.496,0.690,9.983100e-01,0.50,no reported evidence,NaN,4,0,2010.0,2017.0
2,"Acute Myeloid Leukemia, M1",C0026998,IDH1,3417,O75874,"isocitrate dehydrogenase (NADP(+)) 1, cytosolic",NaN,48,0.463,0.690,3.337500e-12,0.32,no reported evidence,1.0,2,0,2011.0,2017.0
3,"Acute Myeloid Leukemia, M1",C0026998,KMT2A,4297,Q03164,lysine methyltransferase 2A,NaN,44,0.460,0.793,1.000000e+00,0.32,no reported evidence,1.0,2,0,1996.0,2016.0
4,"Acute Myeloid Leukemia, M1",C0026998,IDH2,3418,P48735,"isocitrate dehydrogenase (NADP(+)) 2, mitochon...",NaN,92,0.469,0.655,8.652500e-01,0.31,no reported evidence,1.0,2,0,2011.0,2017.0


In [3]:
# basic stats about our dataset

print('Number of genes involved: {}'.format(df['Gene'].nunique()))
print('Number of UniProtAC involved: {}'.format(df['UniProt'].nunique()))

print('Number of null values in UniProt is {}'.format(df['UniProt'].isnull().sum())) # 
print('GeneSymbol for which uniprot entry is null: {}'.format(df[df['UniProt'].isnull()]['Gene'].values))

Number of genes involved: 107
Number of UniProtAC involved: 106
Number of null values in UniProt is 1
GeneSymbol for which uniprot entry is null: ['DLEU2']


In [4]:
# removing the gene that has not no uniprot entry as it is typically found in Mouse
df = df[df.Gene != 'DLEU2']

In [5]:
dff = df[['Gene','Gene_id','UniProt','Gene_Full_Name']]
dff.rename(columns={'Gene':'GeneSymbol','UniProt':'UniProtAC','Gene_id':'GeneID','Gene_Full_Name':'ProteinName'},inplace=True)
dff.to_csv('gene_map.csv',encoding='utf8',index=False)
dff.head()

,GeneSymbol,GeneID,UniProtAC,ProteinName
0,FLT3,2322,P36888,fms related tyrosine kinase 3
1,NPM1,4869,P06748,nucleophosmin 1
2,IDH1,3417,O75874,"isocitrate dehydrogenase (NADP(+)) 1, cytosolic"
3,KMT2A,4297,Q03164,lysine methyltransferase 2A
4,IDH2,3418,P48735,"isocitrate dehydrogenase (NADP(+)) 2, mitochon..."


#### checking the updated symbols and approved seed genes data as per HGNC website

In [6]:
gene_list = dff['GeneSymbol']
print(len(gene_list))
gene_list.to_csv('gene_list.csv', index = False)

106


In [7]:
hgnc = pd.read_csv('hgnc-symbol-check1.csv',skiprows=1)
print(hgnc.shape)
print('The number of approved symbol: {}'.format(len(hgnc[hgnc['Match type'] == 'Approved symbol'])))
print('The number of symbols not approved: {}'.format(len(hgnc[hgnc['Match type'] == 'Previous symbol'])))

print("\n Here's the list of genes that symbols are not approved")
hgnc[hgnc['Match type'] == 'Previous symbol'].head()

(112, 6)
The number of approved symbol: 103
The number of symbols not approved: 3

 Here's the list of genes that symbols are not approved


,Input,Match type,Approved symbol,Approved name,HGNC ID,Location
53,SEPT9,Previous symbol,SEPTIN9,septin 9,HGNC:7323,17q25.3
86,H1F0,Previous symbol,H1-0,H1.0 linker histone,HGNC:4714,22q13.1
87,HIST1H1C,Previous symbol,H1-2,"H1.2 linker histone, cluster member",HGNC:4716,6p22.2


In [8]:
non_approved_genes = hgnc[hgnc['Match type']=='Previous symbol']['Input'].values
approved_name = hgnc[hgnc['Match type']=='Previous symbol']['Approved symbol'].values

for i in range(len(non_approved_genes)):
    dff = dff.replace(non_approved_genes[i],approved_name[i])

print(dff.shape)

(106, 4)


In [9]:
our_uniprot = dff['UniProtAC']
print(len(our_uniprot))
our_uniprot.to_csv('our_uniprot', index = False)

106


#### 1.1 b) Collecting required seed genes information from UniProt website

In [10]:
#raw file from uniprot website using our gene_list
uniprot = pd.read_csv('uniprot-yourlist_M20200108216DA2B77BFBD2E6699CA9B6D1C41EB26F69CBG.tab',sep='\t')
#pre-processing
uniprot.replace({'Function [CC]':'FUNCTION:'},'',regex=True,inplace=True)
uniprot.rename(columns={'Entry':'UniProtAC','Function [CC]':'Description','Gene names  (primary )':'GeneSymbol','Protein names':'ProteinName'},inplace=True)
uniprot['ProteinName'] = uniprot['ProteinName'].str.split('(').str[0]
uniprot['Description'] = uniprot['Description'].astype(str)
uniprot['Description'] = uniprot['Description'].str.split('.').str[0]
uniprot.to_csv('uniprot_final.csv',encoding='utf8',index=False)
print(uniprot.shape)
uniprot.head()

(106, 5)


,UniProtAC,Entry name,GeneSymbol,ProteinName,Description
0,P36888,FLT3_HUMAN,FLT3,Receptor-type tyrosine-protein kinase FLT3,Tyrosine-protein kinase that acts as cell-sur...
1,P06748,NPM_HUMAN,NPM1,Nucleophosmin,Involved in diverse cellular processes such a...
2,O75874,IDHC_HUMAN,IDH1,Isocitrate dehydrogenase [NADP] cytoplasmic,nan
3,Q03164,KMT2A_HUMAN,KMT2A,Histone-lysine N-methyltransferase 2A,Histone methyltransferase that plays an essen...
4,P48735,IDHP_HUMAN,IDH2,"Isocitrate dehydrogenase [NADP], mitochondrial",Plays a role in intermediary metabolism and e...


#### 1.1 c) Store the data gathered in a table in an easily accessible format of your choice (csv, tab, excel, etc).

In [13]:
df_final = pd.merge(dff[['GeneSymbol', 'GeneID', 'UniProtAC', 'ProteinName']],
                    uniprot[['UniProtAC', 'GeneSymbol', 'Description']],
                   on=['GeneSymbol','UniProtAC'])

df_final.to_csv('df_final.csv', encoding='utf8', index=False)
print(len(df_final))
df_final.head()

106


,GeneSymbol,GeneID,UniProtAC,ProteinName,Description
0,FLT3,2322,P36888,fms related tyrosine kinase 3,Tyrosine-protein kinase that acts as cell-sur...
1,NPM1,4869,P06748,nucleophosmin 1,Involved in diverse cellular processes such a...
2,IDH1,3417,O75874,"isocitrate dehydrogenase (NADP(+)) 1, cytosolic",nan
3,KMT2A,4297,Q03164,lysine methyltransferase 2A,Histone methyltransferase that plays an essen...
4,IDH2,3418,P48735,"isocitrate dehydrogenase (NADP(+)) 2, mitochon...",Plays a role in intermediary metabolism and e...


#### List of Seed Genes

In [15]:
seed_genes = list(df_final['GeneSymbol'].unique())
print(len(seed_genes))
print(sorted(seed_genes))

106
['ADCY7', 'AGRN', 'ANXA2', 'ANXA4', 'ANXA5', 'ANXA6', 'AQP9', 'ASMTL', 'ASXL2', 'ATP1B1', 'BAALC', 'BACH2', 'BCL2', 'CAPG', 'CAPN2', 'CASP7', 'CBFB', 'CCND2', 'CD33', 'CD44', 'CD9', 'CDK6', 'CEBPA', 'CEBPD', 'CHMP5', 'CNR2', 'CSF1R', 'CSF2', 'CSF3', 'CST3', 'CTNNA1', 'CTSH', 'CTSZ', 'DAPK1', 'DHX15', 'DNMT3A', 'EHD3', 'EHMT2', 'EIF4EBP1', 'ENAH', 'ENO2', 'ERG', 'FAS', 'FHL2', 'FLT3', 'FOXO1', 'FXYD6', 'GAS2L1', 'GATA2', 'GFI1', 'GTF2I', 'H1-0', 'H1-2', 'HGF', 'HOXA9', 'HSPB1', 'ID2', 'IDH1', 'IDH2', 'IFI30', 'JAK2', 'KMT2A', 'KMT2C', 'KRAS', 'LPAR1', 'LYL1', 'MET', 'MN1', 'MX1', 'MYC', 'MYH11', 'NF1', 'NPM1', 'NRAS', 'NUP98', 'PDE4B', 'PIM2', 'POU4F1', 'PSIP1', 'PTPN11', 'PXDN', 'RASGRP1', 'RGS2', 'RUNX1', 'RUNX1T1', 'RUNX3', 'S100A10', 'S100A8', 'SEPTIN9', 'SGK1', 'SPARC', 'SPI1', 'SPRY4', 'STAT3', 'SVIL', 'SYNGR1', 'TCEA2', 'TNFSF10', 'TRH', 'TRIO', 'TSC2', 'TUBB2A', 'VOPP1', 'VSIG4', 'WT1', 'ZBTB7A']


#### List of Seed Gene IDs

In [16]:
seed_geneIDs = list(df_final['GeneID'].unique())
print(len(seed_genes))
print((seed_geneIDs))

106
[2322, 4869, 3417, 4297, 3418, 4629, 2624, 1050, 6281, 6446, 6678, 6688, 4763, 6774, 6840, 6919, 7200, 6279, 5997, 4330, 4599, 4609, 55252, 4893, 4928, 5142, 5457, 5781, 7204, 7249, 11326, 30845, 51341, 51510, 53826, 55740, 58508, 60468, 79870, 81552, 81848, 11168, 11040, 10919, 7280, 7490, 7837, 8623, 8743, 9145, 10125, 10437, 10634, 10801, 375790, 113, 864, 865, 894, 928, 945, 960, 1021, 1052, 1269, 1436, 862, 861, 840, 302, 307, 308, 309, 355, 366, 481, 596, 822, 824, 1437, 1440, 1471, 2969, 3005, 3006, 3082, 3205, 3315, 3398, 3717, 3845, 4066, 2672, 2308, 2274, 1495, 1512, 1522, 1612, 1665, 1788, 1902, 1978, 2026, 2078, 4233]


### 1.2 COLLECT INTERACTION DATA
- a) Downloading the data from BioGrid database
- b) Store the data gathered from the two DBs in two different tables/matrices in an easily accessible format of your choice.
- c) Summarize the main results in a table reporting

- [Download data from BioGrid Database](https://downloads.thebiogrid.org//BioGRID//Latest-Release// "BioGrid Database")

- FILE NAME : BIOGRID-ALL-LATEST.tab2.zip

<font color='green'> **Loading and cleaning BioGrid database</font>**

In [17]:
""" Loading the Biogrid database from the file and extracting the required columns """
df_biogrid = pd.read_csv('BIOGRID-ALL-3.5.180.tab2.txt', sep='\t')
print('Entire dataset')
print(df_biogrid.shape)

# extract only genes human-human interactions
df_biogrid = df_biogrid.loc[(df_biogrid['Organism Interactor A'] == 9606) & (df_biogrid['Organism Interactor B'] == 9606)]
print('Only Human genes dataset')
print(df_biogrid.shape)

# reduce and rename the columns while resetting the index
df_biogrid = df_biogrid[['Official Symbol Interactor A',
                         'Official Symbol Interactor B']]

df_biogrid.rename(columns={'Official Symbol Interactor A':'GeneA',
                           'Official Symbol Interactor B':'GeneB'}, inplace=True)

df_biogrid.reset_index(inplace=True)
df_biogrid = df_biogrid.drop(columns=['index'])


print('Select only required features')
print(df_biogrid.shape)


print('Convert gene symbols to approved ones ------------------------------------------------')

all_genes_biogrid = list(set(df_biogrid['GeneA'].values)) + list(set(df_biogrid['GeneB'].values))
all_genes_biogrid = list(set(all_genes_biogrid))
print('The number of all genes found in biogrid: ', len(all_genes_biogrid))
all_genes_biogrid = pd.Series(all_genes_biogrid)
all_genes_biogrid.to_csv('1.2. genes_biogrid', index = False)

hgnc = pd.read_csv('1.2. hgnc-symbol-check_biogrid.csv', skiprows=1)
print('The number of approved symbol: {}'.format(len(hgnc[hgnc['Match type'] == 'Approved symbol'])))
print('The number of not approved symbol: {}'.format(len(hgnc[hgnc['Match type'] == 'Previous symbol'])))

non_approved_genes = hgnc[hgnc['Match type'] == 'Previous symbol']['Input'].values
approved_name = hgnc[hgnc['Match type'] == 'Previous symbol']['Approved symbol'].values


# change gene symbols to approved ones
for i in trange(len(non_approved_genes)):
    df_biogrid = df_biogrid.replace(non_approved_genes[i], approved_name[i])


Entire dataset
(1739944, 24)
Only Human genes dataset
(493288, 24)
Select only required features
(493288, 2)
Convert gene symbols to approved ones ------------------------------------------------
The number of all genes found in biogrid:  17913
The number of approved symbol: 17644
The number of not approved symbol: 8


100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 11.53it/s]


#### Number of seed genes found in Biogrid 

In [18]:
# Gathering genes data in the BioGrid database

As = list(df_biogrid.GeneA.values)
Bs = list(df_biogrid.GeneB.values)
all_genes_biogrid = set(As + Bs)
print('No. of all genes found in this database: {}'.format(len(all_genes_biogrid)))

seed_genes_biogrid = list(all_genes_biogrid.intersection(set(seed_genes)))

print('No. of seed genes found: {}'.format(len(seed_genes_biogrid)))
print('Genes not found in the database: {}'.format(set(seed_genes).difference(all_genes_biogrid)))

No. of all genes found in this database: 17909
No. of seed genes found: 100
Genes not found in the database: {'H1-2', 'H1-0', 'TRH', 'CSF3', 'CNR2', 'SEPTIN9'}


#### Total Number of interacting proteins including seed genes

<font color='blue'>**Interactions among the seed genes **</font>

- GeneA AND GeneB

In [19]:
# Number of interactions amongst the seed genes (Among blue nodes and blue line in the diagram)

df_biogrid_seed_genes = df_biogrid.loc[df_biogrid['GeneA'].isin(seed_genes_biogrid) 
                                       & df_biogrid['GeneB'].isin(seed_genes_biogrid)]
print("The Number of interactions between the seed genes",len(df_biogrid_seed_genes))
df_biogrid_seed_genes.head()

The Number of interactions between the seed genes 318


,GeneA,GeneB
193,KMT2A,KMT2A
1287,ASMTL,ASMTL
3888,DNMT3A,MYC
3889,MYC,DNMT3A
3890,DNMT3A,MYC


<font color='blue'>**Interactions with atleast one seed gene or a non seed Gene**</font>

- GeneA OR GeneB

In [20]:
# Number of interactions with atleast one seed genes or non seed genes
#(Between blue and yellow nodes with blue line in the diagram)

df_biogrid_seed_non_seed = df_biogrid.loc[df_biogrid['GeneA'].isin(seed_genes_biogrid) 
                                          | df_biogrid['GeneB'].isin(seed_genes_biogrid)]
print("The Number of interactions between the seed genes or Non seed genes",len(df_biogrid_seed_non_seed))
df_biogrid_seed_non_seed.head()

The Number of interactions between the seed genes or Non seed genes 16570


,GeneA,GeneB
3,GATA2,PML
4,RPA2,STAT3
15,CSF1R,GRB2
80,KDR,ANXA5
87,SUMO1,FAS


<font color='red'> ** Getting the Number of Non seed Genes**</font>

In [21]:
print('The number of seed genes: {}'.format(len(seed_genes_biogrid)))
print('')

df_biogrid_seed = df_biogrid.loc[df_biogrid['GeneA'].isin(seed_genes_biogrid) | df_biogrid['GeneB'].isin(seed_genes_biogrid)]

#Number of non seed genes (red dotted line in the diagram)
non_seed_genesA = set(df_biogrid_seed['GeneA']).difference(set(seed_genes))
non_seed_genesB = set(df_biogrid_seed['GeneB']).difference(set(seed_genes))

non_seed_genes_biogrid  = list(non_seed_genesA.union(non_seed_genesB))
print("Number of non seed genes present in biogrid are ",len(non_seed_genes_biogrid))
print('')
all_genes = non_seed_genes_biogrid + seed_genes_biogrid
print('Total no. of interacting genes (seed genes and non seed genes): {}'.format(len(all_genes)))

The number of seed genes: 100

Number of non seed genes present in biogrid are  5826

Total no. of interacting genes (seed genes and non seed genes): 5926


<font color='red'>**Interactions among Non seed Genes **</font>

In [22]:
# #Number of interactions between the non seed genes (red dotted line in the diagram)
# df_biogrid_non_seed = df_biogrid.loc[df_biogrid['GeneA'].isin(non_seed_genes_biogrid) 
#                                      & df_biogrid['GeneB'].isin(non_seed_genes_biogrid)]
# print("The Number of interactions between the non seed genes A & B :",len(df_biogrid_non_seed))
# df_biogrid_non_seed.head()

In [23]:
#Number of interactions between the non seed genes (red dotted line in the diagram)
df_biogrid_non_seed = df_biogrid.loc[df_biogrid['GeneA'].isin(seed_genes_biogrid) | df_biogrid['GeneB'].isin(seed_genes_biogrid) |\
                                    (df_biogrid['GeneA'].isin(non_seed_genes_biogrid) & df_biogrid['GeneB'].isin(non_seed_genes_biogrid))]
                                     
print("Total Number of interactions  A & B :",len(df_biogrid_non_seed))
df_biogrid_non_seed.head()

Total Number of interactions  A & B : 269952


,GeneA,GeneB
3,GATA2,PML
4,RPA2,STAT3
5,ARF1,GGA3
8,XRN1,ALDOA
9,APP,APPBP2


In [24]:
#Number of interactions between the non seed genes (red dotted line in the diagram)
df_biogrid_final = df_biogrid.loc[df_biogrid['GeneA'].isin(seed_genes_biogrid) | df_biogrid['GeneB'].isin(seed_genes_biogrid) |\
                                    (df_biogrid['GeneA'].isin(non_seed_genes_biogrid) & df_biogrid['GeneB'].isin(non_seed_genes_biogrid))]
                                     
print("Total Number of interactions  A & B :",len(df_biogrid_final))
df_biogrid_final.head()

Total Number of interactions  A & B : 269952


,GeneA,GeneB
3,GATA2,PML
4,RPA2,STAT3
5,ARF1,GGA3
8,XRN1,ALDOA
9,APP,APPBP2


In [27]:
# Storing all the BioGrid interactions data into an easily accessible excel sheet format.

writer_object = pd.ExcelWriter('master_df_biogrid.xlsx',engine='xlsxwriter')

df_biogrid.to_excel(writer_object,sheet_name='df_biogrid',encoding='utf8',index=False)
df_biogrid_seed_genes.to_excel(writer_object,sheet_name='df_biogrid_seed_genes',encoding='utf8',index=False)
df_biogrid_seed_non_seed.to_excel(writer_object,sheet_name='df_biogrid_seed_non_seed',encoding='utf8',index=False)
df_biogrid_non_seed.to_excel(writer_object,sheet_name='df_biogrid_non_seed',encoding='utf8',index=False)
#df_biogrid_atleast_one_non_seed.to_excel(writer_object,sheet_name='df_biogrid_atleast_one_non_seed',encoding='utf8',index=False)

writer_object.save()

df_biogrid.head()

,GeneA,GeneB
0,MAP2K4,FLNC
1,MYPN,ACTN2
2,ACVR1,FNTA
3,GATA2,PML
4,RPA2,STAT3


#### Repeating the above steps for Seed and Non-seed genes in IID Database 

- Getting IID data for the seed genes from the below link
- [Download from IID Database](http://iid.ophid.utoronto.ca/search_by_proteins/)
- Interactions in IID dataset

#### IID Pre-processing

In [29]:
df_iid = pd.read_csv('PPIs.txt',sep='\t')
df_iid = df_iid[['UniProt1','UniProt2','symbol1','symbol2']]
df_iid['symbol1'] = df_iid['symbol1'].str.split(';').str[0]
df_iid['symbol1'] = df_iid['symbol1'].str.split(' ').str[0]
df_iid['symbol2'] = df_iid['symbol2'].str.split(';').str[0]
df_iid['symbol2'] = df_iid['symbol2'].str.split(' ').str[0]
df_iid.to_csv('IIDdata.csv',encoding='utf8',index=False)

df_iid.rename(columns={ 'symbol1':'GeneA',
                                          'symbol2':'GeneB',
                                          'UniProt1':'UniProtA',
                                          'UniProt2':'UniProtB'}, inplace=True)
df_iid.reset_index(inplace=True)
df_iid = df_iid.drop(columns=['index'])

print(df_iid.shape)

print('Convert gene symbols to approved ones ------------------------------------------------')

all_genes_iid = list(set(df_iid['GeneA'].values)) + list(set(df_iid['GeneB'].values))
all_genes_iid = list(set(all_genes_iid))
print('The number of all genes found in IID: ', len(all_genes_iid))
all_genes_iid = pd.Series(all_genes_iid)
all_genes_iid.to_csv('1.2. genes_iid', index = False)

hgnc = pd.read_csv('1.2. hgnc-symbol-check_iid.csv', skiprows=1)
print('The number of approved symbol: {}'.format(len(hgnc[hgnc['Match type'] == 'Approved symbol'])))
print('The number of not approved symbol: {}'.format(len(hgnc[hgnc['Match type'] == 'Previous symbol'])))

non_approved_genes = hgnc[hgnc['Match type'] == 'Previous symbol']['Input'].values
approved_name = hgnc[hgnc['Match type'] == 'Previous symbol']['Approved symbol'].values

for i in range(len(non_approved_genes)):
    df_iid = df_iid.replace(non_approved_genes[i], approved_name[i])
    
df_iid.to_csv('our_iid.csv', encoding='utf8', index=False)

df_iid.head(3)

(8764, 4)
Convert gene symbols to approved ones ------------------------------------------------
The number of all genes found in IID:  4773
The number of approved symbol: 4669
The number of not approved symbol: 126


,UniProtA,UniProtB,GeneA,GeneB
0,O00141,P49411,SGK1,TUFM
1,O00141,Q14697,SGK1,GANAB
2,O00141,Q99759,SGK1,MAP3K3


In [30]:
# Getting the count of all seed genes present in IID database

iid_A = list(df_iid.GeneA.values)
iid_B = list(df_iid.GeneB.values)

all_iid_genes = set(iid_A+iid_B)

print('Total No. of genes found in IID database: {}'.format(len(all_iid_genes)))

seed_genes_iid = list(all_iid_genes.intersection(set(seed_genes)))

print('No. of seed genes found in IID : {}'.format(len(seed_genes_iid)))
print('Genes not found in IID database: {}'.format(set(seed_genes).difference(all_iid_genes)))


Total No. of genes found in IID database: 4759
No. of seed genes found in IID : 106
Genes not found in IID database: set()


<font color='blue'>**Seed Genes Interactions in IID**</font>

In [32]:
# Number of interactions amongst the seed genes (Among blue nodes and blue line in the diagram)
df_iid_seed_genes = df_iid.loc[df_iid['GeneA'].isin(seed_genes_iid) & df_iid['GeneB'].isin(seed_genes_iid)]
print("The Number of interactions between the seed genes",len(df_iid_seed_genes))
df_iid_seed_genes.head()

The Number of interactions between the seed genes 166


,UniProtA,UniProtB,GeneA,GeneB
56,O00141,P49815,SGK1,TSC2
66,O00141,O00141,SGK1,SGK1
160,O43143,P01106,DHX15,MYC
294,O43143,P06748,DHX15,NPM1
384,O60674,P78347,JAK2,GTF2I


<font color='blue'>** Interaction among Seed and Non-seed Genes in IID**</font>

In [33]:
# Number of interactions between the seed genes and non seed genes (Between blue and yellow nodes with blue line in the diagram)
df_iid_seed_non_seed = df_iid.loc[df_iid['GeneA'].isin(seed_genes_iid) | df_iid['GeneB'].isin(seed_genes_iid)]
print("The Number of interactions between the seed genes and Non seed genes",len(df_iid_seed_non_seed))
df_iid_seed_non_seed.head()

The Number of interactions between the seed genes and Non seed genes 8764


,UniProtA,UniProtB,GeneA,GeneB
0,O00141,P49411,SGK1,TUFM
1,O00141,Q14697,SGK1,GANAB
2,O00141,Q99759,SGK1,MAP3K3
3,O00141,P10636,SGK1,MAPT
4,O00141,P04406,SGK1,GAPDH


<font color='red'>**Getting count of Non-seed genes in IID**</font>

In [34]:
print('The number of seed genes: {}'.format(len(seed_genes_iid)))
print('')

df_iid_seed = df_iid.loc[df_iid['GeneA'].isin(seed_genes_iid) | df_iid['GeneB'].isin(seed_genes_iid)]

#Number of non seed genes (red dotted line in the diagram)
non_seed_genesA = set(df_iid_seed['GeneA']).difference(set(seed_genes))
non_seed_genesB = set(df_iid_seed['GeneB']).difference(set(seed_genes))

non_seed_genes_iid  = list(non_seed_genesA.union(non_seed_genesB))
print("Number of non seed genes present in IID are ",len(non_seed_genes_iid))
print('')
all_genes = non_seed_genes_iid + seed_genes_iid
print('Total no. of interacting genes (seed genes and non seed genes): {}'.format(len(all_genes)))


The number of seed genes: 106

Number of non seed genes present in IID are  4653

Total no. of interacting genes (seed genes and non seed genes): 4759


<font color='red'>**Non-seed Interactions in IID**</font>

#### Total Interactions found in IID

In [35]:
df_iid_total_interactions = df_iid.loc[df_iid['GeneA'].isin(seed_genes_biogrid) |  df_iid['GeneB'].isin(seed_genes_biogrid) 
                    | (df_iid['GeneA'].isin(non_seed_genes_iid) & df_iid['GeneB'].isin(non_seed_genes_iid))]

print('Total no. of interactions found: {}'.format(len(df_iid)))

Total no. of interactions found: 8764


In [36]:
df_iid_final = df_iid.loc[df_iid['GeneA'].isin(seed_genes_biogrid) |  df_iid['GeneB'].isin(seed_genes_biogrid) 
                    | (df_iid['GeneA'].isin(non_seed_genes_iid) & df_iid['GeneB'].isin(non_seed_genes_iid))]

print('Total no. of interactions found: {}'.format(len(df_iid_final)))

Total no. of interactions found: 8508


In [37]:
# Storing all the IID interactions data into an easily accessible excel sheet format.

writer_object = pd.ExcelWriter('master_df_iid.xlsx',engine='xlsxwriter')

df_iid.to_excel(writer_object,sheet_name='df_iid',encoding='utf8',index=False)
df_iid_seed_genes.to_excel(writer_object,sheet_name='df_iid_seed_genes',encoding='utf8',index=False)
df_iid_seed_non_seed.to_excel(writer_object,sheet_name='df_iid_seed_non_seed',encoding='utf8',index=False)
df_iid_total_interactions.to_excel(writer_object,sheet_name='df_iid_non_seed',encoding='utf8',index=False)

writer_object.save()

df_iid.head()

,UniProtA,UniProtB,GeneA,GeneB
0,O00141,P49411,SGK1,TUFM
1,O00141,Q14697,SGK1,GANAB
2,O00141,Q99759,SGK1,MAP3K3
3,O00141,P10636,SGK1,MAPT
4,O00141,P04406,SGK1,GAPDH


### 1.3 ARRANGING DATA INTO INTERACTOMES

- <font color='blue'>** A) Seed Genes Interactome**</font>

**interactor A gene symbol, interactor B gene symbol, interactor A Uniprot AC, interactor B Uniprot AC, database source **
 

#### Build and store three tables:
#### First we create map between GeneSymbol and UniProtAC

In [38]:
uniprot_all = pd.read_csv('1.3. uniprot-filtered-organism__Homo+sapiens+(Human)+[9606]_.tab', sep='\t')
uniprot_all.rename(columns={ 'Gene names  (primary )':'GeneSymbol', 'Entry': 'UniProtAC'}, inplace=True)
print('The number of human-genes found in uniprot: ', len(uniprot_all))
uniprot_all.head(3)

gene_uniprot = {}
count = 0
for i in range(len(uniprot_all)):
    contents = uniprot_all['GeneSymbol'].iloc[i]
    
    if not isinstance(contents, float):
        contents = contents.split('; ')
        for content in contents:
            if content not in gene_uniprot.keys():
                gene_uniprot[content] = uniprot_all['UniProtAC'].iloc[i]
    
print('The number of human-genes having uniprotAC: ', len(gene_uniprot))

The number of human-genes found in uniprot:  188349
The number of human-genes having uniprotAC:  26682


In [39]:
# BioGrid
# extract only seed-seed interactions
#df_seedseed_a_biogrid = df_biogrid.loc[df_biogrid['GeneA'].isin(seed_genes_biogrid) & df_biogrid['GeneB'].isin(seed_genes_biogrid)]
df_seedseed_a_biogrid = df_biogrid_final.loc[df_biogrid_final['GeneA'].isin(seed_genes_biogrid) & df_biogrid_final['GeneB'].isin(seed_genes_biogrid)]
df_seedseed_a_biogrid = df_seedseed_a_biogrid.reset_index(drop=True)

UniProtA, UniProtB = [], []
for i in range(len(df_seedseed_a_biogrid)):
    UniProtA.append(gene_uniprot[df_seedseed_a_biogrid['GeneA'].iloc[i]])
    UniProtB.append(gene_uniprot[df_seedseed_a_biogrid['GeneB'].iloc[i]])

df_seedseed_a_biogrid['UniProtA'] = UniProtA
df_seedseed_a_biogrid['UniProtB'] = UniProtB
df_seedseed_a_biogrid['Database'] = 'BioGrid'

df_seedseed_a_biogrid = df_seedseed_a_biogrid.drop_duplicates(subset=['GeneA', 'GeneB'])

print('Number of seed-seed interactions in BioGrid : {}'.format(len(df_seedseed_a_biogrid)))
df_seedseed_a_biogrid.head()

#IID 

# extract only seed-seed interactions
df_seedseed_a_iid = df_iid.loc[df_iid['GeneA'].isin(seed_genes_iid) & df_iid['GeneB'].isin(seed_genes_iid)]

df_seedseed_a_iid['Database'] = 'IID'

df_seedseed_a_iid = df_seedseed_a_iid.drop_duplicates(subset=['GeneA','GeneB'])

print('Number of seed-seed interactions in IID : {}'.format(len(df_seedseed_a_iid)))
df_seedseed_a_iid.head()

# concating both data frames
# All ------------------------------------------------------------------------------------------------------------------------

df_seedseed_a = pd.concat([df_seedseed_a_biogrid, df_seedseed_a_iid])
df_seedseed_a = df_seedseed_a.reset_index(drop=True)

df_seedseed_a.to_csv('1.3_a_seed_seed_interactome_table.csv',encoding='utf8',index=False)

print('Number of seed genes interactome (seed-seed interactions) in both DB : {}'.format(len(df_seedseed_a)))
df_seedseed_a.head(5)


Number of seed-seed interactions in BioGrid : 182
Number of seed-seed interactions in IID : 162
Number of seed genes interactome (seed-seed interactions) in both DB : 344


,Database,GeneA,GeneB,UniProtA,UniProtB
0,BioGrid,KMT2A,KMT2A,Q03164,Q03164
1,BioGrid,ASMTL,ASMTL,O95671,O95671
2,BioGrid,DNMT3A,MYC,Q9Y6K1,P01106
3,BioGrid,MYC,DNMT3A,P01106,Q9Y6K1
4,BioGrid,CTNNA1,CTNNA1,P35221,P35221


- <font color='blue'>** B) Union Interactome**</font>

All proteins interacting with at least one seed gene,from all DBs, same format as above.

**interactor A gene symbol, interactor B gene symbol, interactor A Uniprot AC, interactor B Uniprot AC, database source **

In [40]:
# biogrid --------------------------------------------------------------------------------------------------------------------------
 # extract interactions with at least one seed gene
#df_union_b_biogrid = df_biogrid.loc[df_biogrid['GeneA'].isin(seed_genes_biogrid) | df_biogrid['GeneB'].isin(seed_genes_biogrid)]
df_union_b_biogrid = df_biogrid_final.copy()
df_union_b_biogrid = df_union_b_biogrid.reset_index(drop=True)

UniProtA, UniProtB = [], []
non_approved_genes = []
for i in range(len(df_union_b_biogrid)):
    geneA = df_union_b_biogrid['GeneA'].iloc[i]
    geneB= df_union_b_biogrid['GeneB'].iloc[i]
    try:
        uniprotA = gene_uniprot[geneA]
        uniprotB = gene_uniprot[geneB]
        
        UniProtA.append(uniprotA)
        UniProtB.append(uniprotB)
        
    except:
        non_approved_genes.append(i)

# dropping the non_approved genes        
df_union_b_biogrid = df_union_b_biogrid.drop(non_approved_genes)

df_union_b_biogrid['UniProtA'] = UniProtA
df_union_b_biogrid['UniProtB'] = UniProtB
df_union_b_biogrid['Database'] = 'BioGrid'

df_union_b_biogrid = df_union_b_biogrid.drop_duplicates(subset=['GeneA', 'GeneB'])

# print('Some genes are removed since they do not exist on UniProt')
print('Number of genes that do not exist in UniProt:{}'.format(len(non_approved_genes)))
print('Number of interactions in BioGrid : {} '.format(len(df_union_b_biogrid)))

# iid --------------------------------------------------------------------------------------------------------------------------
 # extract interactions with at least one seed gene
df_union_b_iid = df_iid.loc[df_iid['GeneA'].isin(seed_genes_iid) | df_iid['GeneB'].isin(seed_genes_iid)]
df_union_b_iid['Database'] = 'IID'

df_union_b_iid = df_union_b_iid.drop_duplicates(subset=['GeneA', 'GeneB'])

print('Number of interactions in IID : {}'.format(len(df_union_b_iid)))

# All ------------------------------------------------------------------------------------------------------------------------
df_union_b = pd.concat([df_union_b_biogrid, df_union_b_iid])
df_union_b = df_union_b.reset_index(drop=True)

df_union_b.to_csv('1.3_b_union_interactome_table.csv',encoding='utf8',index=False)

print('Number of interactions in union interactome in both DB : {}'.format(len(df_union_b)))
df_union_b.head(3)

Number of genes that do not exist in UniProt:9487
Number of interactions in BioGrid : 198286 
Number of interactions in IID : 8748
Number of interactions in union interactome in both DB : 207034


,Database,GeneA,GeneB,UniProtA,UniProtB
0,BioGrid,GATA2,PML,P23769,P29590
1,BioGrid,RPA2,STAT3,P15927,P40763
2,BioGrid,ARF1,GGA3,P84077,Q9NZ52


- <font color='blue'>** C) Intersection Interactome**</font>

All proteins interacting with at least one seed gene confirmed

**interactor A gene symbol, interactor B gene symbol, interactor A Uniprot AC, interactor B Uniprot AC, database source **

In [41]:
# Obtain intersection (drop duplicate rows in terms of feartures['GeneA', 'GeneB'])
# It went well because I already dropped duplicate rows before concatenating [df_intersection_b_biogrid & df_intersection_b_iid ]
df_intersection_c = df_union_b[df_union_b.duplicated(['GeneA', 'GeneB'])]
df_intersection_c = df_intersection_c.drop('Database', axis = 1)
df_intersection_c = df_intersection_c.reset_index(drop=True)

df_intersection_c.to_csv('1.3_c_intersection_interactome_table.csv',encoding='utf8',index=False)

print('Number of intersection interactome in both DB : {}'.format(len(df_intersection_c)))
df_intersection_c.head(3)

Number of intersection interactome in both DB : 3943


,GeneA,GeneB,UniProtA,UniProtB
0,SGK1,TUFM,O00141,P49411
1,SGK1,MAPT,O00141,P10636
2,SGK1,CDKN1B,O00141,P46527


In [42]:
# Storing all the Interactomes to an easily accessible excel sheet format.

writer_object = pd.ExcelWriter('master_df_interactomes.xlsx',engine='xlsxwriter')

df_seedseed_a.to_excel(writer_object,sheet_name='df_seedseed_interactome_a',encoding='utf8',index=False)
df_union_b.to_excel(writer_object,sheet_name='df_union_interactome_b',encoding='utf8',index=False)
df_intersection_c.to_excel(writer_object,sheet_name='df_intersection_interactome_c',encoding='utf8',index=False)

writer_object.save()

## 1.4 Enrichment analysis
Using the service [Enrichr](https://amp.pharm.mssm.edu/Enrichr/), find, report in tables and save related charts (8 charts in total) of the overrepresented GO categories (limit to the first 10 for each main category, BP, MF, CL) and the the overrepresented pathways (KEGG 2019 Human) for:

- a) the seed genes
- b) the union interactome genes

### a) Seed Genes

In [43]:
df_seedseed_a = df_seedseed_a.drop_duplicates(['GeneA', 'GeneB'])
df_seedseed_a = df_seedseed_a.drop('Database', axis = 1)
df_seedseed_a = df_seedseed_a.reset_index(drop=True)
print('Number of interactions in seed interactome in both DB : {}'.format(len(df_seedseed_a)))

geneA = list(df_seedseed_a['GeneA'].values)
geneB = list(df_seedseed_a['GeneB'].values)
genes = list(set(geneA + geneB))
genes = pd.Series(genes)
genes.to_csv('1.4. seed_genes.csv', index = False)
print('Number of seed genes found in both DB : {}'.format(len(genes)))

Number of interactions in seed interactome in both DB : 256
Number of seed genes found in both DB : 83


In [60]:
import os
path = r'C:\Users\alevi\DS_HOME\BioInformatics\Project2\1.4 EnrichmentAnalysis\GO_Analysis\seed_seed'

In [61]:
# GO Biological Process 2018
seed_enricher_BP = pd.read_csv(os.path.join(path , 'GO_Biological_Process_2018_table.txt'),sep='\t')
seed_enricher_BP = seed_enricher_BP[['Term','Overlap','P-value','Adjusted P-value','Odds Ratio','Combined Score']]
seed_enricher_BP.to_csv('1.4_a_GO_Biological_Process_2018_table.csv',encoding='utf8',index=False)
seed_enricher_BP.head(10)

,Term,Overlap,P-value,Adjusted P-value,Odds Ratio,Combined Score
0,regulation of apoptotic process (GO:0042981),23/815,1.435630e-13,7.326018e-10,6.800207,201.095739
1,negative regulation of programmed cell death (...,17/408,7.843325e-13,2.001224e-09,10.040161,279.858870
2,regulation of myeloid cell differentiation (GO...,9/65,6.186446e-12,1.052314e-08,33.364226,861.085980
3,negative regulation of apoptotic process (GO:0...,17/485,1.225554e-11,1.563500e-08,8.446156,212.210029
4,cytokine-mediated signaling pathway (GO:0019221),18/633,8.774059e-11,8.954805e-08,6.852053,158.670495
5,negative regulation of cell proliferation (GO:...,13/363,3.150347e-09,2.679370e-06,8.629560,168.930129
6,cellular response to cytokine stimulus (GO:007...,14/456,5.296174e-09,3.860911e-06,7.398013,140.978617
7,regulation of cell proliferation (GO:0042127),17/740,7.885399e-09,5.029899e-06,5.535656,103.285673
8,"positive regulation of transcription, DNA-temp...",20/1120,2.173566e-08,1.232412e-05,4.302926,75.922167
9,positive regulation of peptidyl-tyrosine phosp...,8/116,2.754785e-08,1.405767e-05,16.618197,289.278629


In [62]:
#GO_Molecular_Function_2018
seed_enricher_MF = pd.read_csv(os.path.join(path , 'GO_Molecular_Function_2018_table.txt'),sep='\t')
seed_enricher_MF = seed_enricher_MF[['Term','Overlap','P-value','Adjusted P-value','Odds Ratio','Combined Score']]
seed_enricher_MF.to_csv('1.4_a_GO_Molecular_Function_2018_table.csv',encoding='utf8',index=False)
seed_enricher_MF.head(10)

,Term,Overlap,P-value,Adjusted P-value,Odds Ratio,Combined Score
0,C2H2 zinc finger domain binding (GO:0070742),3/12,0.000015,0.016986,60.240964,670.106201
1,protein homodimerization activity (GO:0042803),12/664,0.000018,0.010480,4.354768,47.526086
2,transcription regulatory region DNA binding (G...,9/374,0.000025,0.009465,5.798595,61.522855
3,protein tyrosine kinase activity (GO:0004713),6/147,0.000034,0.009704,9.835259,101.276900
4,"transcriptional activator activity, RNA polyme...",7/284,0.000179,0.041146,5.939250,51.253177
5,protein kinase activity (GO:0004672),9/513,0.000273,0.052452,4.227436,34.683915
6,regulatory region DNA binding (GO:0000975),6/224,0.000340,0.055858,6.454389,51.553894
7,phospholipase inhibitor activity (GO:0004859),2/8,0.000469,0.067444,60.240964,461.771114
8,histone-lysine N-methyltransferase activity (G...,3/41,0.000656,0.083847,17.631502,129.237517
9,core promoter binding (GO:0001047),4/97,0.000713,0.082020,9.936654,72.006898


In [64]:
#GO_Cellular_Component_2018
seed_enricher_CC = pd.read_csv(os.path.join(path , 'GO_Cellular_Component_2018_table.txt'),sep='\t')
seed_enricher_CC = seed_enricher_CC[['Term','Overlap','P-value','Adjusted P-value','Odds Ratio','Combined Score']]
seed_enricher_CC.to_csv('1.4_a_GO_Cellular_Component_2018_table.csv',encoding='utf8',index=False)
seed_enricher_CC.head(10)

,Term,Overlap,P-value,Adjusted P-value,Odds Ratio,Combined Score
0,focal adhesion (GO:0005925),15/356,1.729093e-11,7.711753e-09,10.152971,251.599153
1,chromatin (GO:0000785),9/296,3.802348e-06,8.479236e-04,7.326604,91.435221
2,nuclear periphery (GO:0034399),5/78,1.813888e-05,2.696647e-03,15.446401,168.635352
3,nuclear matrix (GO:0016363),4/59,1.054787e-04,1.176088e-02,16.336533,149.593650
4,membrane raft (GO:0045121),5/119,1.374077e-04,1.225676e-02,10.124532,90.032989
5,nuclear chromatin (GO:0000790),6/253,6.462466e-04,4.803766e-02,5.714558,41.969595
6,secretory granule lumen (GO:0034774),6/317,2.052377e-03,1.307657e-01,4.560830,28.225868
7,platelet alpha granule membrane (GO:0031092),2/17,2.222324e-03,1.238946e-01,28.348689,173.187863
8,tertiary granule (GO:0070820),4/164,4.845662e-03,2.401295e-01,5.877167,31.323370
9,nuclear chromosome part (GO:0044454),6/392,5.797451e-03,2.585663e-01,3.688222,18.995587


In [67]:
# KEGG 2019 HUMAN
kegg_human_seed_enrichr = pd.read_csv(os.path.join(path,'KEGG_2019_Human_table.txt'),sep='\t')
kegg_human_seed_enrichr = kegg_human_seed_enrichr[['Term','Overlap','P-value','Adjusted P-value','Odds Ratio','Combined Score']]
kegg_human_seed_enrichr.to_csv('1.4_a_KEGG_2019_Human_table.csv',encoding='utf8',index=False)
kegg_human_seed_enrichr.head(10)

,Term,Overlap,P-value,Adjusted P-value,Odds Ratio,Combined Score
0,Pathways in cancer,23/530,1.416777e-17,4.363672e-15,10.456922,405.682169
1,Acute myeloid leukemia,12/66,4.681260e-17,7.209141e-15,43.811610,1647.333154
2,Transcriptional misregulation in cancer,16/186,4.888894e-17,5.019265e-15,20.728074,778.483854
3,PI3K-Akt signaling pathway,15/354,1.596539e-11,1.229335e-09,10.210333,253.834979
4,Apoptosis,8/143,1.407830e-07,8.672232e-06,13.480495,212.668919
5,Proteoglycans in cancer,9/201,1.512087e-07,7.762045e-06,10.789426,169.443680
6,Hematopoietic cell lineage,7/97,1.557689e-07,6.853833e-06,17.389144,272.572961
7,Central carbon metabolism in cancer,6/65,2.888038e-07,1.111895e-05,22.242817,334.921628
8,Cellular senescence,8/160,3.338168e-07,1.142395e-05,12.048193,179.670765
9,JAK-STAT signaling pathway,8/162,3.670589e-07,1.130542e-05,11.899450,176.322991


### b) Union Interactome Genes

In [68]:
df_union_b = df_union_b.drop_duplicates(subset=['GeneA', 'GeneB'])
#df_union_b = df_union_b.drop('Database', axis = 1)
df_union_b = df_union_b.reset_index(drop=True)
print('Number of interactions in union interactome in both DB : {}'.format(len(df_union_b)))

geneA = list(df_union_b['GeneA'].values)
geneB = list(df_union_b['GeneB'].values)
genes = list(set(geneA + geneB))
genes = pd.Series(genes)
genes.to_csv('1.4. union_interacome_genes.csv', index = False)
print('Number of seed genes found in both DB : {}'.format(len(genes)))

Number of interactions in union interactome in both DB : 203091
Number of seed genes found in both DB : 6621


In [75]:
# enrichment analysis for union interactome
path = r'C:\Users\alevi\DS_HOME\BioInformatics\Project2\1.4 EnrichmentAnalysis\GO_Analysis\union_interactome'
union_enricher_BP = pd.read_csv(os.path.join(path , 'GO_Biological_Process_2018_table.txt'),sep='\t')
union_enricher_BP = union_enricher_BP[['Term','Overlap','P-value','Adjusted P-value','Odds Ratio','Combined Score']]
union_enricher_BP.to_csv('1.4_b_GO_Biological_Process_2018_table.csv',encoding='utf8',index=False)
union_enricher_BP.head(10)

,Term,Overlap,P-value,Adjusted P-value,Odds Ratio,Combined Score
0,regulation of transcription from RNA polymeras...,812/1478,4.234328e-72,2.160777e-68,1.659541,272.733795
1,positive regulation of gene expression (GO:001...,488/771,1.768360e-68,4.511970e-65,1.911929,298.271949
2,"positive regulation of transcription, DNA-temp...",644/1120,1.069545e-66,1.819297e-63,1.736898,263.840643
3,regulation of apoptotic process (GO:0042981),498/815,1.085115e-62,1.384336e-59,1.845772,263.352199
4,"negative regulation of transcription, DNA-temp...",487/813,1.669707e-57,1.704103e-54,1.809443,236.556946
5,gene expression (GO:0010467),293/411,1.721155e-57,1.463842e-54,2.153437,281.463617
6,ribosome biogenesis (GO:0042254),189/226,6.355211e-56,4.632949e-53,2.526154,321.062744
7,rRNA metabolic process (GO:0016072),172/200,1.248564e-54,7.964276e-52,2.597795,322.432071
8,rRNA processing (GO:0006364),172/202,2.666323e-53,1.511805e-50,2.572074,311.365805
9,ncRNA processing (GO:0034470),186/227,2.405500e-52,1.227527e-49,2.475104,294.182641


In [76]:
union_enricher_MF = pd.read_csv(os.path.join(path , 'GO_Molecular_Function_2018_table.txt'),sep='\t')
union_enricher_MF = union_enricher_MF[['Term','Overlap','P-value','Adjusted P-value','Odds Ratio','Combined Score']]
union_enricher_MF.to_csv('1.4_b_GO_Molecular_Function_2018_table.csv',encoding='utf8',index=False)
union_enricher_MF.head(10)

,Term,Overlap,P-value,Adjusted P-value,Odds Ratio,Combined Score
0,RNA binding (GO:0003723),919/1387,2.199402e-151,2.531512e-148,2.001453,694.308475
1,cadherin binding (GO:0045296),249/313,7.639852e-65,4.396735e-62,2.403042,354.772317
2,protein kinase binding (GO:0019901),333/495,1.020128e-55,3.913893e-53,2.032102,257.309295
3,protein kinase activity (GO:0004672),329/513,5.596061e-48,1.610267e-45,1.937247,210.776362
4,kinase binding (GO:0019900),277/418,1.624994e-44,3.740735e-42,2.001750,201.832957
5,transcription regulatory region DNA binding (G...,247/374,2.081648e-39,3.993294e-37,1.994949,177.685421
6,ubiquitin-like protein ligase binding (GO:0044...,203/297,7.953371e-36,1.307761e-33,2.064648,166.863743
7,protein serine/threonine kinase activity (GO:0...,237/368,3.208229e-35,4.615839e-33,1.945391,154.512223
8,transcription coactivator activity (GO:0003713),199/291,3.429370e-35,4.385784e-33,2.065696,163.929746
9,ubiquitin protein ligase binding (GO:0031625),191/284,2.569341e-32,2.957311e-30,2.031522,147.770996


In [77]:
union_enricher_CC = pd.read_csv(os.path.join(path , 'GO_Cellular_Component_2018_table.txt'),sep='\t')
union_enricher_CC = union_enricher_CC[['Term','Overlap','P-value','Adjusted P-value','Odds Ratio','Combined Score']]
union_enricher_CC.to_csv('1.4_b_GO_Cellular_Component_2018_table.csv',encoding='utf8',index=False)
union_enricher_CC.head(10)

,Term,Overlap,P-value,Adjusted P-value,Odds Ratio,Combined Score
0,focal adhesion (GO:0005925),288/356,3.863285e-78,1.723025e-75,2.443706,435.590829
1,nucleolus (GO:0005730),449/676,1.618671e-72,3.609635e-70,2.006347,331.658235
2,nuclear body (GO:0016604),375/618,4.022621e-46,5.980296e-44,1.832944,191.592104
3,chromatin (GO:0000785),203/296,3.721673e-36,4.149666e-34,2.071623,169.000709
4,nuclear chromosome part (GO:0044454),243/392,3.231631e-32,2.882615e-30,1.872521,135.775977
5,nucleoplasm part (GO:0044451),248/407,3.612529e-31,2.685313e-29,1.840618,129.019466
6,cytoskeleton (GO:0005856),297/520,4.705731e-30,2.998223e-28,1.725280,116.506019
7,cytosolic part (GO:0044445),121/159,9.044053e-29,5.042059e-27,2.298765,148.437858
8,nuclear speck (GO:0016607),187/296,1.618878e-26,8.022442e-25,1.908342,113.327828
9,microtubule cytoskeleton (GO:0015630),229/388,4.205394e-26,1.875606e-24,1.782831,104.172318


In [78]:
# KEGG 2019 HUMAN
kegg_human_union_enrichr = pd.read_csv(os.path.join(path,'KEGG_2019_Human_table.txt'),sep='\t')
kegg_human_union_enrichr = kegg_human_union_enrichr[['Term','Overlap','P-value','Adjusted P-value','Odds Ratio','Combined Score']]
kegg_human_union_enrichr.to_csv('1.4_b_KEGG_2019_Human_table.csv',encoding='utf8',index=False)
kegg_human_union_enrichr.head(10)

,Term,Overlap,P-value,Adjusted P-value,Odds Ratio,Combined Score
0,Pathways in cancer,332/530,4.042148e-45,1.244981e-42,1.892207,193.420544
1,Human T-cell leukemia virus 1 infection,163/219,3.168471e-36,4.879445e-34,2.248277,183.773778
2,Cellular senescence,126/160,1.570885e-32,1.612775e-30,2.378795,174.201718
3,Apoptosis,116/143,2.701171e-32,2.079902e-30,2.450351,178.113680
4,Cell cycle,104/124,1.597316e-31,9.839468e-30,2.533483,179.653901
5,Proteoglycans in cancer,143/201,1.849967e-28,9.496499e-27,2.149049,137.232305
6,Epstein-Barr virus infection,142/201,9.274629e-28,4.080837e-26,2.134021,132.832352
7,Hepatitis B,121/163,4.770796e-27,1.836757e-25,2.242354,135.902986
8,Chronic myeloid leukemia,69/76,8.277786e-26,2.832842e-24,2.742470,158.387624
9,Ubiquitin mediated proteolysis,105/137,1.386732e-25,4.271134e-24,2.315129,132.512590


# Part 2 - Data analysis ------------------------------------------------------

### Scope of the project:
Starting from **the seed genes interactome (SGI), the intersection (I) and the union (U)** interactomes built in the first part of the project, 
- compute the main network measures for I, U and their nodes, 
- apply clustering methods for disease modules discovery, 
- carry on an enrichment analysis on the putative disease modules and 
- produce a short report.


In [53]:
SGI  = df_seedseed_a[['GeneA', 'GeneB']]
print(len(SGI))
SGI.to_csv('2. SGI.csv', index = False)

U = df_union_b[['GeneA', 'GeneB']]
print(len(U))
U.to_csv('2. U.csv', index = False)

I = df_intersection_c[['GeneA', 'GeneB']]
print(len(I))
I.to_csv('2. I.csv', index = False)

256
203091
3943


## 2.1) Calculate the main network measures for SGI, I and U

### Calculate the main network measures for SGI, I and U
#### a) Calculate the following global (i.e. concerning the whole network and not the single nodes)
measures of SGI, U and I (only if no. of nodes >20):

- No. of nodes and no. of links
-  No. of connected components
- No. of isolated nodes
- Average path length
- Average degree
- Average clustering coefficient
- Network diameter & radius
- Centralization


### Feeding the Genes data from the interactomes into  CytoScape for our Analysis
Downloading the required statistics from cytoscape
Now we have files named (***2.1. result_SGI.netstats, 2.1. result_I.netstats, 2.1. result_U.netstats***)

In [83]:
#loading the files downloaded from cytoscapes
stats = ['interactome', 'nodeCount', 'nsl', 'links', 'ncc', 'usn', 'avSpl', 'cc', 'avNeighbors', 'diameter', 'radius','centralization']
measure = pd.DataFrame(columns=stats)
interactomes = ['SGI', 'I', 'U']

for idx, interactome in enumerate(interactomes):
    measure.set_value(idx, 'interactome', interactome)
    
    path = r'C:\Users\alevi\DS_HOME\BioInformatics\Project2\2.1_Data_analysis'
    filename = os.path.join(path,'2.1. stats_' + interactome + '.netstats')
    file = open(filename, "r")
    
    for row in file:
        row = row.split(' ')
        stat = row[0]

        if stat in stats:
            value = row[-1].split('\n')[0]
            value = round(float(value), 3)
            measure.set_value(idx, stat, value)

# We use networkx for centralization
import networkx as nx

G_SGI = nx.from_pandas_edgelist(SGI, 'GeneA', 'GeneB')
G_I = nx.from_pandas_edgelist(I, 'GeneA', 'GeneB')
G_U = nx.from_pandas_edgelist(U, 'GeneA', 'GeneB')

def centralization(G):
    n=G.number_of_nodes()
    centralities=nx.degree_centrality(G).values()
    Centralization=(n*max(centralities) - sum(centralities))/ (n-1)**2
    return round(Centralization, 5)

measure.set_value([0,1,2], 'centralization', [centralization(G_SGI), centralization(G_I), centralization(G_U)])

measure.set_value([0,1,2], 'links', [len(SGI), len(I), len(U)])


,interactome,nodeCount,nsl,links,ncc,usn,avSpl,cc,avNeighbors,diameter,radius,centralization
0,SGI,83,60,256,14,12,3.872,0.125,3.566,9,1,0.00346
1,I,2677,38,3943,4,0,3.58,0.014,2.917,9,1,7e-05
2,U,6621,1638,203091,1,0,3.073,0.098,55.148,7,1,5e-05


### Global 

#### b) Isolate the largest connected component (LCC) of I and U and calculate the following global and local (i.e. for each node) measures:
i)
- N. of nodes and no. of links
- Average path length
- Average degree
- Average clustering coefficient
- Network diameter & radius
- Centralization

#### I-LCC 

In [86]:
path = r'C:\Users\alevi\DS_HOME\BioInformatics\Project2\2.1_Data_analysis'
I_LCC = pd.read_csv(os.path.join(path,'2.1. I_LCC default  edge.csv'))
links = len(I_LCC)
geneA, geneB = [],[]
for i in range(len(I_LCC)):
    A = I_LCC.iloc[i]['name'].split(' (interacts with) ')[0]
    B = I_LCC.iloc[i]['name'].split(' (interacts with) ')[1]
    geneA.append(A)
    geneB.append(B)
    
I_LCC['GeneA'] = geneA
I_LCC['GeneB'] = geneB
I_LCC = I_LCC[['GeneA', 'GeneB']]
print(len(I_LCC))

measure.set_value(3, 'interactome', 'I-LCC')
filename = os.path.join(path,'2.1. stats_I_LCC.netstats')
file = open(filename, "r")

for row in file:
    row = row.split(' ')
    stat = row[0]

    if stat in stats:
        value = row[-1].split('\n')[0]
        value = round(float(value), 3)
        measure.set_value(3, stat, value)

measure.set_value(3, 'links', links)

# centralization -------------------------------------------------
G_I_LCC = nx.from_pandas_edgelist(I_LCC, 'GeneA', 'GeneB')

measure.set_value(3, 'centralization', centralization(G_I_LCC))


3895


,interactome,nodeCount,nsl,links,ncc,usn,avSpl,cc,avNeighbors,diameter,radius,centralization
0,SGI,83,60,256,14,12,3.872,0.125,3.566,9,1,0.00346
1,I,2677,38,3943,4,0,3.58,0.014,2.917,9,1,7e-05
2,U,6621,1638,203091,1,0,3.073,0.098,55.148,7,1,5e-05
3,I-LCC,2664,0,3895,1,0,3.58,0.014,2.923,9,1,7e-05


#### b) Isolate the largest connected component (LCC) of I and U and calculate the following global and local (i.e. for each node) measures:
i)
- N. of nodes and no. of links
- Average path length
- Average degree
- Average clustering coefficient
- Network diameter & radius
- Centralization

#### U-LCC

In our case, U-LCC is the same network as U because U has only one component

In [87]:
measure = measure.append(measure.iloc[2])
measure = measure.reset_index(drop=True)
measure.set_value(4, 'interactome', 'U-LCC')

,interactome,nodeCount,nsl,links,ncc,usn,avSpl,cc,avNeighbors,diameter,radius,centralization
0,SGI,83,60,256,14,12,3.872,0.125,3.566,9,1,0.00346
1,I,2677,38,3943,4,0,3.58,0.014,2.917,9,1,7e-05
2,U,6621,1638,203091,1,0,3.073,0.098,55.148,7,1,5e-05
3,I-LCC,2664,0,3895,1,0,3.58,0.014,2.923,9,1,7e-05
4,U-LCC,6621,1638,203091,1,0,3.073,0.098,55.148,7,1,5e-05


In [173]:
measure = measure.rename({'nodeCount':'nodes', 
                          'nsl': 'self-loops', 
                          'ncc': 'connected components', 
                          'usn': 'isolated nodes', 
                          'avSpl': 'average path length', 
                          'cc': 'average clustering coefficient', 
                          'avNeighbors': 'average degree',
                          'centralization': 'centralization'}, axis = 1)
measure.to_csv('2.1_SGI_I_U_whole_network.csv',encoding='utf8',index=False)
measure

,interactome,nodes,self-loops,links,connected components,isolated nodes,average path length,average clustering coefficient,average degree,diameter,radius,centralization
0,SGI,83,60,256,14,12,3.872,0.125,3.566,9,1,0.00346
1,I,2677,38,3943,4,0,3.58,0.014,2.917,9,1,7e-05
2,U,6621,1638,203091,1,0,3.073,0.098,55.148,7,1,5e-05
3,I-LCC,2664,0,3895,1,0,3.58,0.014,2.923,9,1,7e-05
4,U-LCC,6621,1638,203091,1,0,3.073,0.098,55.148,7,1,5e-05


### Local 

ii)
- Node degree
- Betweenness centrality
- Eigenvector centrality
- Closeness centrality
- ratio Betweenness/Node degree


#### I-LCC Network

In [177]:
path = r'C:\Users\alevi\DS_HOME\BioInformatics\Project2\2.1_Data_analysis'
I_LCC = pd.read_csv(os.path.join(path,'2.1. I_LCC default  node.csv'))
I_LCC = I_LCC[['name', 'Indegree', 'Outdegree', 'BetweennessCentrality', 'ClosenessCentrality']]
I_LCC['ratio Betweenness/Nodedegree'] = I_LCC['BetweennessCentrality'] / ( I_LCC['Indegree'] + I_LCC['Outdegree'])
# Eigenvector Centrality --------------------
eigen = nx.eigenvector_centrality_numpy(G_I_LCC)
genes = list(eigen.keys())
eigenvalue = list(eigen.values())
eigen_centrality = pd.DataFrame(columns=['name', 'EigenvectorCentrality'])
eigen_centrality['name'] = genes
eigen_centrality['EigenvectorCentrality'] = eigenvalue

I_LCC = pd.merge(I_LCC, eigen_centrality, on=['name'])
I_LCC = I_LCC.sort_values(by=['BetweennessCentrality'], ascending=False)[:20]
I_LCC = I_LCC[['name', 'Indegree', 'Outdegree', 'BetweennessCentrality', 'EigenvectorCentrality','ClosenessCentrality','ratio Betweenness/Nodedegree']]
I_LCC = I_LCC.reset_index(drop=True)
I_LCC.to_csv(os.path.join(path,'2.2_I_LCC_Highest_20_betweeness.csv'),encoding='utf8',index=False)
I_LCC

,name,Indegree,Outdegree,BetweennessCentrality,EigenvectorCentrality,ClosenessCentrality,ratio Betweenness/Nodedegree
0,STAT3,74,84,0.006425,0.060636,0.340852,0.000041
1,NPM1,27,383,0.004870,0.313752,0.440842,0.000012
2,MYC,16,508,0.003766,0.599906,0.446479,0.000007
3,HSPB1,17,250,0.002885,0.141267,0.367162,0.000011
4,CEBPA,15,44,0.002366,0.051102,0.354046,0.000040
5,CDK6,22,62,0.002269,0.050420,0.405242,0.000027
6,PTPN11,71,40,0.002179,0.021083,0.536232,0.000020
7,MAPK8,3,2,0.001950,0.030569,0.280262,0.000390
8,RUNX1,25,24,0.001766,0.062564,0.437888,0.000036
9,FHL2,47,29,0.001695,0.044542,0.525773,0.000022


#### U-LCC Network

In [178]:
path = r'C:\Users\alevi\DS_HOME\BioInformatics\Project2\2.1_Data_analysis'
U_LCC = pd.read_csv(os.path.join(path, '2.1. U default  node.csv'))
U_LCC = U_LCC.sort_values(by=['BetweennessCentrality'], ascending=False)[:20]
U_LCC = U_LCC[['name', 'Indegree', 'Outdegree', 'BetweennessCentrality', 'ClosenessCentrality']]
U_LCC['ratio Betweenness/Nodedegree'] = U_LCC['BetweennessCentrality'] / ( U_LCC['Indegree'] + U_LCC['Outdegree'])

# Eigenvector Centrality --------------------
eigen = nx.eigenvector_centrality_numpy(G_U)
genes = list(eigen.keys())
eigenvalue = list(eigen.values())
eigen_centrality = pd.DataFrame(columns=['name', 'EigenvectorCentrality'])
eigen_centrality['name'] = genes
eigen_centrality['EigenvectorCentrality'] = eigenvalue

U_LCC = pd.merge(U_LCC, eigen_centrality, on=['name'])
U_LCC = U_LCC.sort_values(by=['BetweennessCentrality'], ascending=False)[:20]
U_LCC = U_LCC[['name', 'Indegree', 'Outdegree', 'BetweennessCentrality', 'EigenvectorCentrality','ClosenessCentrality','ratio Betweenness/Nodedegree']]
U_LCC = U_LCC.reset_index(drop=True)
U_LCC.to_csv(os.path.join(path,'2.2_U_LCC_Highest_20_betweeness.csv'),encoding='utf8',index=False)
U_LCC

,name,Indegree,Outdegree,BetweennessCentrality,EigenvectorCentrality,ClosenessCentrality,ratio Betweenness/Nodedegree
0,MYC,266,2190,0.073403,0.139624,0.568414,0.000030
1,APP,830,200,0.045589,0.050414,0.433620,0.000044
2,KRAS,60,1547,0.026270,0.067963,0.524812,0.000016
3,TP53,453,635,0.024469,0.074395,0.489375,0.000022
4,NPM1,280,629,0.019164,0.076411,0.465607,0.000021
5,HSPB1,209,436,0.016570,0.038566,0.454877,0.000026
6,ELAVL1,123,828,0.016001,0.060041,0.502967,0.000017
7,STAT3,250,255,0.015236,0.021401,0.440136,0.000030
8,TRIM25,92,1199,0.014806,0.090511,0.518984,0.000011
9,PTPN11,221,173,0.014308,0.015533,0.419621,0.000036


## 2.2) Apply clustering methods for disease modules discovery

Cluster I-LCC and U-LCC using the **MCL** algorithm to get the modules.

Once you have clustered the networks, find modules with no. of nodes >= 10 in which seed genes are statistically overrepresented (p < 0.05) by applying a hypergeometric test: such modules will be the “**putative disease modules**”.

Store the results for both U-LCC and I-LCC in tables including in each row: clustering algorithm used, module ID, no. of seed genes in the module, total no. of genes in each module, seed gene IDs, all gene IDs in the module, p-value.

### I-LCC

In [91]:
I_LCC_MCL = pd.read_csv(os.path.join(path,'2.2. I_LCC--clustered default  node.csv'))
print('The number of genes found in this network:', len(I_LCC_MCL))
print('The number of seed genes found in this network:', len(set(I_LCC_MCL['name'].values).intersection(seed_genes)))
I_LCC_MCL.head()

The number of genes found in this network: 2664
The number of seed genes found in this network: 93


,SUID,__mclCluster,name,selected,shared name
0,4076,1.0,TIMP3,False,TIMP3
1,4074,1.0,GTPBP2,False,GTPBP2
2,4072,1.0,KRTAP4-12,False,KRTAP4-12
3,4069,1.0,ELP3,False,ELP3
4,4067,1.0,LLGL2,False,LLGL2


In [92]:
print('I-LCC')
clusters_I = {}
for moduleID in range(1, 30):
    current_df = I_LCC_MCL[I_LCC_MCL['__mclCluster'] == moduleID]
    
    if len(current_df) < 10: # only the cluster having at least 10 genes
        break
        
    clusters_I[moduleID] = list(current_df.name.values)
    
num_clusters = len(clusters_I.keys())
num_genes_I_LCC = len(I_LCC_MCL)
print('The number of clusters having more than 10 genes: {}'.format(num_clusters))
print('The number of genes in all clusters: {}'.format(num_genes_I_LCC))

I-LCC
The number of clusters having more than 10 genes: 15
The number of genes in all clusters: 2664


#### mapping gene-ID (used in 2.2)

In [95]:
df_mapping =  pd.read_csv(os.path.join(path,'2.2. mapping.csv'))
df_mapping.dropna(inplace=True)
df_mapping['GeneID']=df_mapping['GeneID'].astype('int')
mapping = dict(zip(df_mapping.Gene, df_mapping.GeneID))
print(len(mapping)) #18194

18198


In [187]:
# deleting the Nan
print(df_mapping.isna().sum())
del mapping['Uncharacterized'], mapping['SMAP'], mapping['Tcr-alpha'], mapping['cDNA']

Gene      0
GeneID    0
dtype: int64


In [141]:
mapping = dict(zip(df_mapping.Gene, df_mapping.GeneID))

In [99]:
from scipy.stats import hypergeom

M = num_genes_I_LCC
N = len(set(I_LCC_MCL['name'].values).intersection(seed_genes))
output = {}

for moduleID, genes in clusters_I.items():
    module_genes = genes
    module_genes_ID = [mapping[i] for i in module_genes]
    n = len(module_genes_ID)

    seed_genes_in_cluster = list(set(module_genes).intersection(seed_genes))
    seed_genes_in_cluster_ID = [mapping[i] for i in seed_genes_in_cluster]
    k = len(seed_genes_in_cluster_ID)
    
    p_value = hypergeom.sf(k, M, n, N)

    output[moduleID]  = [seed_genes_in_cluster_ID, module_genes_ID, p_value]
    
putative_I_LCC = {}
for i in list(output.keys()):
        if output[i][2] < 0.05:
            putative_I_LCC[i] = output[i]
            
print('the number of putative disease modules', len(putative_I_LCC))

the number of putative disease modules 6


In [100]:
### U-LCC

U_LCC_MCL = pd.read_csv(os.path.join(path,'2.2. U--clustered default  node.csv'))
print('The number of genes found in this network:', len(U_LCC_MCL))
print('The number of seed genes found in this network:', len(set(U_LCC_MCL['name'].values).intersection(seed_genes)))
U_LCC_MCL.head()

The number of genes found in this network: 6621
The number of seed genes found in this network: 106


,SUID,__mclCluster,name,selected,shared name
0,223772,1.0,IL17A,False,IL17A
1,223744,3.0,ADGRL1,False,ADGRL1
2,223711,3.0,UPK3BL1,False,UPK3BL1
3,223701,3.0,ABHD18,False,ABHD18
4,223689,34.0,SEPTIN8,False,SEPTIN8


In [101]:
print('U-LCC')
clusters_U = {}
for moduleID in range(1, 30):
    current_df = U_LCC_MCL[U_LCC_MCL['__mclCluster'] == moduleID]
    
    if len(current_df) < 10:
        break
        
    clusters_U[moduleID] = list(current_df.name.values)
        
num_clusters = len(clusters_U.keys())
num_genes_U_LCC = len(U_LCC_MCL)
print('The number of clusters having more than 10 genes: {}'.format(num_clusters))
print('The number of genes in all clusters: {}'.format(num_genes_U_LCC))

U-LCC
The number of clusters having more than 10 genes: 29
The number of genes in all clusters: 6621


In [102]:
M = num_genes_U_LCC
N = len(set(U_LCC_MCL['name'].values).intersection(seed_genes))
output = {}

for moduleID, genes in clusters_U.items():
    module_genes = genes
    module_genes_ID = [mapping[i] for i in module_genes]
    n = len(module_genes_ID)

    seed_genes_in_cluster = list(set(module_genes).intersection(seed_genes))
    seed_genes_in_cluster_ID = [mapping[i] for i in seed_genes_in_cluster]
    k = len(seed_genes_in_cluster_ID)
    
    p_value = hypergeom.sf(k, M, n, N)

    output[moduleID]  = [seed_genes_in_cluster_ID, module_genes_ID, p_value]
    
putative_U_LCC = {}
for i in list(output.keys()):
        if output[i][2] < 0.05:
            putative_U_LCC[i] = output[i]
print('the number of putative disease modules:', len(putative_U_LCC))

the number of putative disease modules: 2


In [104]:
df_putative = pd.DataFrame(columns=['interactome',  'algorithm', 'module ID', 
                                    'no. of seed genes'  ,'total no. of genes','ratio seed/total',
                                    'seed gene IDs', 'all gene IDs', 'p-value'])


count = 0
# I-LCC ------------------------------------------------------------------------------------
for moduleID,v in putative_I_LCC.items():
    df_putative.set_value(count, 'interactome', 'I-LCC')
    df_putative.set_value(count, 'algorithm', 'Marcov')
    df_putative.set_value(count, 'module ID', moduleID)
    df_putative.set_value(count, 'no. of seed genes', len(v[0]))
    df_putative.set_value(count, 'total no. of genes', len(v[1]))
    df_putative.set_value(count, 'ratio seed/total', round(len(v[0])/len(v[1]),5))
    df_putative.set_value(count, 'seed gene IDs', ','.join(str(int(v)) for v in v[0]))
    df_putative.set_value(count, 'all gene IDs', ','.join(str(int(v)) for v in v[1]))
    df_putative.set_value(count, 'p-value', v[2])
    count += 1
    
# U-LCC ------------------------------------------------------------------------------------
for moduleID,v in putative_U_LCC.items():
    df_putative.set_value(count, 'interactome', 'U-LCC')
    df_putative.set_value(count, 'algorithm', 'Marcov')
    df_putative.set_value(count, 'module ID', moduleID)
    df_putative.set_value(count, 'no. of seed genes', len(v[0]))
    df_putative.set_value(count, 'total no. of genes', len(v[1]))
    df_putative.set_value(count, 'ratio seed/total', round(len(v[0])/len(v[1]),5))
    df_putative.set_value(count, 'seed gene IDs', v[0])
    df_putative.set_value(count, 'seed gene IDs', ','.join(str(int(v)) for v in v[0]))
    df_putative.set_value(count, 'all gene IDs', ','.join(str(int(v)) for v in v[1]))
    df_putative.set_value(count, 'p-value', v[2])
    count += 1

df_putative.to_csv(os.path.join(path,'2.2. putative.csv'),index=False)
df_putative

,interactome,algorithm,module ID,no. of seed genes,total no. of genes,ratio seed/total,seed gene IDs,all gene IDs,p-value
0,I-LCC,Marcov,1,54,904,0.05973,"6840,4869,2969,5997,7280,8623,864,4629,6279,10...","7078,54676,83755,55140,3993,64854,29934,23196,...",3.63107e-07
1,I-LCC,Marcov,4,3,36,0.08333,"3418,307,308","4357,6609,310,307,131578,49860,9374,10319,2222...",0.0349096
2,I-LCC,Marcov,6,2,22,0.09091,"1978,6446","3182,5578,9113,472,1977,6788,699,208,4043,2317...",0.03923
3,I-LCC,Marcov,13,1,10,0.1,824,"637,471,2762,9601,7284,6635,4287,5798,9474,824",0.0452104
4,I-LCC,Marcov,14,1,10,0.1,53826,"5501,221074,56928,9564,722,55142,79939,53826,4...",0.0452104
5,I-LCC,Marcov,15,1,10,0.1,4893,"5910,6224,3337,5294,673,65244,5290,79621,4893,...",0.0452104
6,U-LCC,Marcov,15,2,35,0.05714,"3417,2026","51166,84173,441282,140458,654364,231,9380,5019...",0.0180074
7,U-LCC,Marcov,24,2,27,0.07407,"4330,1052","4330,25932,23234,55689,7264,252969,51056,55818...",0.00882408


## 2.3) Carry on an enrichment analysis on the disease modules

Find overrepresented GO categories (limit to first ten) and overrepresented pathways (limit to first ten) for the genes belonging to each putative disease module.


In [165]:
# Extracting Gene List for I_LCC and U_LCC

path = r'C:\Users\alevi\DS_HOME\BioInformatics\Project2\2.3 EnrichmentAnalysis'
genes_I_LCC_1 = df_putative.iloc[0]['all gene IDs'].split(',')
genes_I_LCC_1 = [mapping2[int(i)] for i in genes_I_LCC_1]
pd.Series(genes_I_LCC_1).to_csv(os.path.join(path,'2.3. genes_I_LCC.csv'), index = False)

genes_U_LCC_1 = df_putative.iloc[6]['all gene IDs'].split(',')
genes_U_LCC_1 = [mapping2[int(i)] for i in genes_U_LCC_1]
pd.Series(genes_U_LCC_1).to_csv(os.path.join(path,'2.3. genes_U_LCC.csv'), index = False)


In [169]:
path = r'C:\Users\alevi\DS_HOME\BioInformatics\Project2\2.3 EnrichmentAnalysis\Enrichr_I_LCC'

# GO Analysis I_LCC

# GO Biological Process 2018
I_LCC_BP = pd.read_csv(os.path.join(path , 'GO_Biological_Process_2018_table.txt'),sep='\t')
I_LCC_BP = I_LCC_BP[['Term','Overlap','P-value','Adjusted P-value','Odds Ratio','Combined Score']]
I_LCC_BP.to_csv(os.path.join(path ,'2.3_a_GO_Biological_Process_2018_table.csv'),encoding='utf8',index=False)
I_LCC_BP.head(10)

#GO_Molecular_Function_2018
I_LCC_MF = pd.read_csv(os.path.join(path , 'GO_Molecular_Function_2018_table.txt'),sep='\t')
I_LCC_MF = I_LCC_MF[['Term','Overlap','P-value','Adjusted P-value','Odds Ratio','Combined Score']]
I_LCC_MF.to_csv(os.path.join(path ,'2.3_a_GO_Molecular_Function_2018_table.csv'),encoding='utf8',index=False)
I_LCC_MF.head(10)

#GO_Cellular_Component_2018
I_LCC_CC = pd.read_csv(os.path.join(path , 'GO_Cellular_Component_2018_table.txt'),sep='\t')
I_LCC_CC = I_LCC_CC[['Term','Overlap','P-value','Adjusted P-value','Odds Ratio','Combined Score']]
I_LCC_CC.to_csv(os.path.join(path ,'2.3_a_GO_Cellular_Component_2018_table.csv'),encoding='utf8',index=False)
I_LCC_CC.head(10)

# Pathway Analysis I_LCC

# KEGG 2019 HUMAN
kegg_I_LCC_enrichr = pd.read_csv(os.path.join(path,'KEGG_2019_Human_table.txt'),sep='\t')
kegg_I_LCC_enrichr = kegg_I_LCC_enrichr[['Term','Overlap','P-value','Adjusted P-value','Odds Ratio','Combined Score']]
kegg_I_LCC_enrichr.to_csv(os.path.join(path ,'2.3_a_KEGG_2019_Human_table.csv'),encoding='utf8',index=False)
kegg_I_LCC_enrichr.head(10)


,Term,Overlap,P-value,Adjusted P-value,Odds Ratio,Combined Score
0,Pathways in cancer,94/530,7.483322e-31,2.304863e-28,3.923860,272.188234
1,Cell cycle,39/124,1.067451e-22,1.643874e-20,6.958321,352.032605
2,Viral carcinogenesis,46/201,3.400096e-20,3.490765e-18,5.063180,226.971703
3,Human T-cell leukemia virus 1 infection,48/219,3.556681e-20,2.738644e-18,4.849073,217.155408
4,Hepatitis B,41/163,9.399751e-20,5.790247e-18,5.564906,243.804192
5,PI3K-Akt signaling pathway,61/354,1.164505e-19,5.977794e-18,3.812309,166.204568
6,Human cytomegalovirus infection,48/225,1.180034e-19,5.192148e-18,4.719764,205.704184
7,FoxO signaling pathway,36/132,9.330817e-19,3.592364e-17,6.033789,250.497552
8,Human papillomavirus infection,57/330,1.701279e-18,5.822154e-17,3.821400,156.353153
9,Hepatitis C,37/155,3.841833e-17,1.183285e-15,5.281188,199.618311


In [168]:
# GO Analysis for U_LCC

path = r'C:\Users\alevi\DS_HOME\BioInformatics\Project2\2.3 EnrichmentAnalysis\Enrichr_U_LCC'

U_LCC_BP = pd.read_csv(os.path.join(path , 'GO_Biological_Process_2018_table.txt'),sep='\t')
U_LCC_BP = U_LCC_BP[['Term','Overlap','P-value','Adjusted P-value','Odds Ratio','Combined Score']]
U_LCC_BP.to_csv(os.path.join(path ,'2.3_b_GO_Biological_Process_2018_table.csv'),encoding='utf8',index=False)
U_LCC_BP.head(10)

U_LCC_MF = pd.read_csv(os.path.join(path , 'GO_Molecular_Function_2018_table.txt'),sep='\t')
U_LCC_MF = U_LCC_MF[['Term','Overlap','P-value','Adjusted P-value','Odds Ratio','Combined Score']]
U_LCC_MF.to_csv(os.path.join(path ,'2.3_b_GO_Molecular_Function_2018_table.csv'),encoding='utf8',index=False)
U_LCC_MF.head(10)

U_LCC_CC = pd.read_csv(os.path.join(path , 'GO_Cellular_Component_2018_table.txt'),sep='\t')
U_LCC_CC = U_LCC_CC[['Term','Overlap','P-value','Adjusted P-value','Odds Ratio','Combined Score']]
U_LCC_CC.to_csv(os.path.join(path ,'2.3_b_GO_Cellular_Component_2018_table.csv'),encoding='utf8',index=False)
U_LCC_CC.head(10)

# Pathway Analysis

# KEGG 2019 HUMAN
kegg_U_LCC_enrichr = pd.read_csv(os.path.join(path,'KEGG_2019_Human_table.txt'),sep='\t')
kegg_U_LCC_enrichr = kegg_U_LCC_enrichr[['Term','Overlap','P-value','Adjusted P-value','Odds Ratio','Combined Score']]
kegg_U_LCC_enrichr.to_csv(os.path.join(path ,'2.3_b_KEGG_2019_Human_table.csv'),encoding='utf8',index=False)
kegg_U_LCC_enrichr.head(10)

,Term,Overlap,P-value,Adjusted P-value,Odds Ratio,Combined Score
0,Glycolysis / Gluconeogenesis,4/68,0.000006,0.001820,33.613445,404.671558
1,HIF-1 signaling pathway,4/100,0.000027,0.004211,22.857143,240.158046
2,Glutathione metabolism,3/56,0.000128,0.013111,30.612245,274.463473
3,Butanoate metabolism,2/28,0.001093,0.084153,40.816327,278.323580
4,Citrate cycle (TCA cycle),2/30,0.001255,0.077304,38.095238,254.501736
5,Glyoxylate and dicarboxylate metabolism,2/30,0.001255,0.064420,38.095238,254.501736
6,"Glycine, serine and threonine metabolism",2/40,0.002226,0.097930,28.571429,174.505437
7,Tryptophan metabolism,2/42,0.002451,0.094380,27.210884,163.566858
8,Fatty acid degradation,2/44,0.002688,0.091974,25.974026,153.743529
9,"Valine, leucine and isoleucine degradation",2/48,0.003191,0.098270,23.809524,136.846417



## 2.4) Find putative disease genes using the DIAMOnD tool

Using the tool DIAMOnD, compute the putative disease protein list using as reference interactome (“network_file”) the whole BioGrid interactome already used to collect PPIs. As “seed_file” use your seed gene list, limit the number of putative disease proteins (“n”) to 200, and omit the “alpha” parameter (it will be set by default to 1).

Software and instruction for [DIAMOnD](https://github.com/barabasilab/DIAMOnD)

Find overrepresented GO categories (limit to first ten) and overrepresented pathways (limit to first ten) of such 200 newly found genes.


In [108]:
df_biogrid = pd.read_csv('BIOGRID-ALL-3.5.180.tab2.txt', sep='\t')
df_biogrid = df_biogrid.loc[(df_biogrid['Organism Interactor A'] == 9606) & (df_biogrid['Organism Interactor B'] == 9606)]
network_file = df_biogrid[['Entrez Gene Interactor A', 'Entrez Gene Interactor B']]

network_file.to_csv('DIAMOnD_network_file.txt', index = False, header=False)
network_file.head()

,Entrez Gene Interactor A,Entrez Gene Interactor B
0,6416,2318
1,84665,88
2,90,2339
3,2624,5371
4,6118,6774


In [109]:
# extracting the seed list to perform diamond analysis
pd.Series(seed_geneIDs).to_csv('DIAMOnD_seed_file.txt', index = False)

In [142]:
# creating a map dictionary for reverse look-up
mapping2 = dict([(value, key) for key, value in mapping.items()])

In [188]:
# The First 30 Genes from Diamond Analysis
path = r'C:\Users\alevi\DS_HOME\BioInformatics\Project2\DIAMOnD'
df_diamond = pd.read_csv(os.path.join(path,'first_200_added_nodes_weight_1.txt'), sep='\t')
df_diamond.rename(columns={'#rank':'Rank'},inplace=True)
df_diamond['Gene'] = df_diamond['DIAMOnD_node'].map(mapping2)
df_diamond['Gene'].to_csv(os.path.join(path,'2.4_diamond_genes.csv'),encoding='utf8',index=False)
df_diamond.to_csv(os.path.join(path,'2.4_First_30_genes.csv'),encoding='utf8',index=False)
df_diamond.head(30)

,Rank,DIAMOnD_node,Gene
0,1,5925,RB1
1,2,5894,RAF1
2,3,5371,PML
3,4,84662,GLIS2
4,5,1969,EPHA2
5,6,405,ARNT
6,7,1030,CDKN2B
7,8,5607,MAP2K5
8,9,2264,FGFR4
9,10,1019,CDK4


<font color='red'> ---- THE  END ---- </font>